# 性能优化实战教程

## 📋 教程概述

本教程专注于视觉语言模型的性能优化,涵盖推理加速、显存优化、批处理等关键技术。

**优化目标**:
- ⚡ 提升推理速度 (2-5x)
- 💾 降低显存占用 (30-50%)
- 📊 提高吞吐量 (3-10x)
- 🎯 保持模型精度

---

## 🎯 学习目标

完成本教程后,您将学会:
1. ✅ 模型量化技术 (INT8/FP16/BF16)
2. ✅ 批处理优化策略
3. ✅ KV Cache优化
4. ✅ Flash Attention加速
5. ✅ 混合精度训练
6. ✅ 性能profiling和分析

---

## 📚 前置要求

- Python 3.8+
- PyTorch 2.0+
- CUDA 11.7+ (推荐)
- 16GB+ GPU显存
- 理解模型推理基础

---

## 📖 教程结构

1. **基准测试** - 建立性能基线
2. **精度优化** - 量化和混合精度
3. **显存优化** - Gradient Checkpointing和KV Cache
4. **推理优化** - 批处理和并发
5. **高级优化** - Flash Attention和编译器优化
6. **性能分析** - Profiling和瓶颈识别
7. **最佳实践** - 优化策略总结

**预计完成时间**: 45-60分钟

## 1. 环境准备

In [ ]:
# 安装依赖
!pip install torch torchvision transformers pillow datasets accelerate bitsandbytes flash-attn --upgrade -q

print("✅ 依赖安装完成")

In [ ]:
import os
import time
import torch
import numpy as np
from PIL import Image
from pathlib import Path
from typing import List, Dict, Optional, Callable
from dataclasses import dataclass
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm
from transformers import (
    AutoModel,
    AutoProcessor,
    BitsAndBytesConfig,
    AutoModelForImageTextToText
)
import warnings
warnings.filterwarnings('ignore')

# 检查环境
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"支持BFloat16: {torch.cuda.is_bf16_supported()}")

## 2. 基准测试

### 2.1 性能测试工具

In [ ]:
@dataclass
class BenchmarkResult:
    """性能测试结果"""
    name: str
    avg_latency: float  # 平均延迟(秒)
    throughput: float   # 吞吐量(samples/sec)
    memory_allocated: float  # 显存分配(GB)
    memory_reserved: float   # 显存保留(GB)
    peak_memory: float       # 峰值显存(GB)
    
    def __str__(self):
        return f"""
📊 {self.name}
  ⏱️  平均延迟: {self.avg_latency:.3f}s
  🚀 吞吐量: {self.throughput:.2f} samples/s
  💾 显存分配: {self.memory_allocated:.2f} GB
  📦 显存保留: {self.memory_reserved:.2f} GB
  ⬆️  峰值显存: {self.peak_memory:.2f} GB
"""

class PerformanceBenchmark:
    """性能基准测试工具"""
    
    def __init__(self, device: str = "cuda"):
        self.device = device
        self.results: List[BenchmarkResult] = []
    
    def benchmark(
        self,
        name: str,
        func: Callable,
        num_runs: int = 10,
        warmup_runs: int = 3
    ) -> BenchmarkResult:
        """执行性能测试"""
        if self.device == "cuda":
            torch.cuda.empty_cache()
            torch.cuda.reset_peak_memory_stats()
        
        # 预热
        for _ in range(warmup_runs):
            func()
        
        if self.device == "cuda":
            torch.cuda.synchronize()
        
        # 测试
        latencies = []
        for _ in tqdm(range(num_runs), desc=f"测试 {name}"):
            start = time.perf_counter()
            func()
            if self.device == "cuda":
                torch.cuda.synchronize()
            end = time.perf_counter()
            latencies.append(end - start)
        
        avg_latency = np.mean(latencies)
        throughput = 1.0 / avg_latency
        
        # 显存统计
        if self.device == "cuda":
            memory_allocated = torch.cuda.memory_allocated() / 1024**3
            memory_reserved = torch.cuda.memory_reserved() / 1024**3
            peak_memory = torch.cuda.max_memory_allocated() / 1024**3
        else:
            memory_allocated = memory_reserved = peak_memory = 0.0
        
        result = BenchmarkResult(
            name=name,
            avg_latency=avg_latency,
            throughput=throughput,
            memory_allocated=memory_allocated,
            memory_reserved=memory_reserved,
            peak_memory=peak_memory
        )
        
        self.results.append(result)
        print(result)
        return result
    
    def compare_results(self):
        """对比所有结果"""
        if not self.results:
            print("❌ 没有测试结果")
            return
        
        # 创建对比表格
        df = pd.DataFrame([
            {
                "方法": r.name,
                "延迟(s)": f"{r.avg_latency:.3f}",
                "吞吐量": f"{r.throughput:.2f}",
                "显存(GB)": f"{r.peak_memory:.2f}"
            }
            for r in self.results
        ])
        
        print("\n📊 性能对比:")
        print(df.to_string(index=False))
        
        # 可视化
        self._plot_comparison()
    
    def _plot_comparison(self):
        """可视化对比"""
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        names = [r.name for r in self.results]
        latencies = [r.avg_latency for r in self.results]
        throughputs = [r.throughput for r in self.results]
        memories = [r.peak_memory for r in self.results]
        
        # 延迟对比
        axes[0].bar(names, latencies, color='skyblue')
        axes[0].set_title('平均延迟对比')
        axes[0].set_ylabel('时间(秒)')
        axes[0].tick_params(axis='x', rotation=45)
        
        # 吞吐量对比
        axes[1].bar(names, throughputs, color='lightgreen')
        axes[1].set_title('吞吐量对比')
        axes[1].set_ylabel('samples/sec')
        axes[1].tick_params(axis='x', rotation=45)
        
        # 显存对比
        axes[2].bar(names, memories, color='coral')
        axes[2].set_title('峰值显存对比')
        axes[2].set_ylabel('显存(GB)')
        axes[2].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

print("✅ 性能测试工具定义完成")

### 2.2 准备测试数据

In [ ]:
# 创建示例图像
def create_test_image(size=(224, 224)):
    """创建测试图像"""
    return Image.new('RGB', size, color=(73, 109, 137))

# 准备测试数据
test_image = create_test_image()
test_prompt = "Describe this image in detail."

print("✅ 测试数据准备完成")
test_image

## 3. 精度优化

### 3.1 基线模型 (Float32)

In [ ]:
# 配置
model_name = "OpenGVLab/InternVL2-2B"  # 使用较小模型演示
device = "cuda" if torch.cuda.is_available() else "cpu"

# 初始化benchmark
benchmark = PerformanceBenchmark(device)

# 加载Float32模型
print("🔄 加载Float32基线模型...")
model_fp32 = AutoModelForImageTextToText.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    trust_remote_code=True
).to(device)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

print("✅ Float32模型加载完成")

In [ ]:
# 基线测试
def inference_fp32():
    inputs = processor(images=test_image, text=test_prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model_fp32.generate(**inputs, max_new_tokens=50)
    return outputs

result_fp32 = benchmark.benchmark("Float32 基线", inference_fp32, num_runs=5)

### 3.2 Float16优化

In [ ]:
# 清理显存
del model_fp32
torch.cuda.empty_cache()

# 加载Float16模型
print("🔄 加载Float16模型...")
model_fp16 = AutoModelForImageTextToText.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to(device)

print("✅ Float16模型加载完成")

In [ ]:
# Float16测试
def inference_fp16():
    inputs = processor(images=test_image, text=test_prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model_fp16.generate(**inputs, max_new_tokens=50)
    return outputs

result_fp16 = benchmark.benchmark("Float16", inference_fp16, num_runs=5)

### 3.3 BFloat16优化

In [ ]:
if torch.cuda.is_available() and torch.cuda.is_bf16_supported():
    # 清理显存
    del model_fp16
    torch.cuda.empty_cache()
    
    # 加载BFloat16模型
    print("🔄 加载BFloat16模型...")
    model_bf16 = AutoModelForImageTextToText.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    ).to(device)
    
    print("✅ BFloat16模型加载完成")
    
    # BFloat16测试
    def inference_bf16():
        inputs = processor(images=test_image, text=test_prompt, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model_bf16.generate(**inputs, max_new_tokens=50)
        return outputs
    
    result_bf16 = benchmark.benchmark("BFloat16", inference_bf16, num_runs=5)
else:
    print("⚠️ 当前GPU不支持BFloat16,跳过测试")

### 3.4 INT8量化

In [ ]:
try:
    # 清理显存
    if 'model_bf16' in locals():
        del model_bf16
    torch.cuda.empty_cache()
    
    # INT8量化配置
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0
    )
    
    print("🔄 加载INT8量化模型...")
    model_int8 = AutoModelForImageTextToText.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        trust_remote_code=True,
        device_map="auto"
    )
    
    print("✅ INT8模型加载完成")
    
    # INT8测试
    def inference_int8():
        inputs = processor(images=test_image, text=test_prompt, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model_int8.generate(**inputs, max_new_tokens=50)
        return outputs
    
    result_int8 = benchmark.benchmark("INT8量化", inference_int8, num_runs=5)
    
except Exception as e:
    print(f"⚠️ INT8量化失败: {e}")

## 4. 批处理优化

### 4.1 单样本 vs 批处理

In [ ]:
# 使用FP16模型进行批处理测试
if 'model_int8' in locals():
    del model_int8
torch.cuda.empty_cache()

# 重新加载FP16模型
model = AutoModelForImageTextToText.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to(device)

# 准备批量数据
batch_sizes = [1, 2, 4, 8]
test_images = [create_test_image() for _ in range(8)]
test_prompts = ["Describe this image." for _ in range(8)]

for batch_size in batch_sizes:
    def inference_batch():
        images = test_images[:batch_size]
        prompts = test_prompts[:batch_size]
        
        inputs = processor(images=images, text=prompts, return_tensors="pt", padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=50)
        return outputs
    
    benchmark.benchmark(f"Batch={batch_size}", inference_batch, num_runs=5)

## 5. KV Cache优化

In [ ]:
# 不使用KV Cache
def inference_no_cache():
    inputs = processor(images=test_image, text=test_prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, use_cache=False)
    return outputs

result_no_cache = benchmark.benchmark("无KV Cache", inference_no_cache, num_runs=5)

# 使用KV Cache
def inference_with_cache():
    inputs = processor(images=test_image, text=test_prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
    return outputs

result_with_cache = benchmark.benchmark("使用KV Cache", inference_with_cache, num_runs=5)

## 6. 性能分析

### 6.1 对比所有结果

In [ ]:
# 显示完整对比
benchmark.compare_results()

### 6.2 计算加速比

In [ ]:
# 计算相对基线的改进
if benchmark.results:
    baseline = benchmark.results[0]  # Float32基线
    
    print("\n🚀 优化效果总结:\n")
    for result in benchmark.results[1:]:
        speedup = baseline.avg_latency / result.avg_latency
        memory_saving = (baseline.peak_memory - result.peak_memory) / baseline.peak_memory * 100
        
        print(f"📊 {result.name}:")
        print(f"  ⚡ 速度提升: {speedup:.2f}x")
        print(f"  💾 显存节省: {memory_saving:.1f}%")
        print()

## 7. 最佳实践总结

### 7.1 优化策略决策树

In [ ]:
optimization_guide = """
## 🎯 优化策略决策树

### 场景1: 显存不足
优先级顺序:
1. ✅ INT8量化 (节省50%+显存)
2. ✅ Gradient Checkpointing
3. ✅ 减小batch size
4. ✅ 使用更小的模型

### 场景2: 推理延迟高
优先级顺序:
1. ✅ BFloat16/Float16 (2x加速)
2. ✅ 启用KV Cache (1.5-2x加速)
3. ✅ Flash Attention (1.2-1.5x加速)
4. ✅ 增大batch size

### 场景3: 吞吐量低
优先级顺序:
1. ✅ 批处理 (3-10x提升)
2. ✅ 异步推理
3. ✅ 模型并行
4. ✅ 多GPU部署

### 场景4: 精度要求高
推荐配置:
- 使用BFloat16 (保持精度)
- 避免INT8量化
- 使用全精度训练

### 场景5: 边缘设备部署
推荐配置:
- INT8量化必选
- 使用小模型 (1-2B)
- 启用所有优化

## 📊 性能优化效果预期

| 优化方法 | 速度提升 | 显存节省 | 精度影响 |
|---------|---------|---------|----------|
| Float16 | 2x | 50% | 微小 |
| BFloat16 | 2x | 50% | 几乎无 |
| INT8 | 1.5-2x | 75% | 小 |
| KV Cache | 1.5-2x | +20% | 无 |
| Batch=4 | 3x | +50% | 无 |
| Batch=8 | 5x | +100% | 无 |
| Flash Attention | 1.2-1.5x | -10% | 无 |
"""

print(optimization_guide)

### 7.2 实用代码模板

In [ ]:
# 最佳实践:组合多种优化
class OptimizedInference:
    """优化的推理类"""
    
    def __init__(
        self,
        model_name: str,
        device: str = "cuda",
        use_fp16: bool = True,
        use_int8: bool = False,
        batch_size: int = 4
    ):
        self.device = device
        self.batch_size = batch_size
        
        # 精度配置
        if use_int8:
            quantization_config = BitsAndBytesConfig(load_in_8bit=True)
            self.model = AutoModelForImageTextToText.from_pretrained(
                model_name,
                quantization_config=quantization_config,
                device_map="auto",
                trust_remote_code=True
            )
        else:
            dtype = torch.float16 if use_fp16 else torch.float32
            self.model = AutoModelForImageTextToText.from_pretrained(
                model_name,
                torch_dtype=dtype,
                trust_remote_code=True
            ).to(device)
        
        self.processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
        self.model.eval()
    
    def __call__(self, images, prompts):
        """批量推理"""
        # 自动批处理
        results = []
        for i in range(0, len(images), self.batch_size):
            batch_images = images[i:i+self.batch_size]
            batch_prompts = prompts[i:i+self.batch_size]
            
            inputs = self.processor(
                images=batch_images,
                text=batch_prompts,
                return_tensors="pt",
                padding=True
            )
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=512,
                    use_cache=True  # 启用KV Cache
                )
            
            batch_results = self.processor.batch_decode(outputs, skip_special_tokens=True)
            results.extend(batch_results)
        
        return results

print("✅ 优化推理类定义完成")

# 使用示例
print("\n📝 使用示例:")
print("""
# 初始化优化推理器
inference = OptimizedInference(
    model_name="OpenGVLab/InternVL2-2B",
    use_fp16=True,
    batch_size=4
)

# 批量推理
images = [Image.open(path) for path in image_paths]
prompts = ["Describe this image." for _ in images]
results = inference(images, prompts)
""")

## 8. 总结

### 8.1 关键收获

In [ ]:
print("""
🎉 性能优化实战完成!

✅ 掌握技能:
1. 模型量化 (INT8/FP16/BF16)
2. 批处理优化
3. KV Cache使用
4. 性能基准测试
5. 显存优化策略

📊 典型优化效果:
- 速度: 2-5x提升
- 显存: 30-75%节省
- 吞吐量: 3-10x提升

🎯 最佳实践:
- 优先使用BFloat16/Float16
- 启用KV Cache
- 合理设置batch size
- 显存不足时考虑INT8

💡 下一步:
1. 在实际项目中应用优化
2. 针对特定场景调优
3. 监控生产环境性能
4. 持续优化和改进
""")

### 8.2 练习任务

1. **基础练习**:
   - 测试不同batch size的性能
   - 对比不同模型大小的优化效果
   - 记录优化前后的详细数据

2. **进阶练习**:
   - 实现动态batch size调整
   - 添加更多profiling指标
   - 尝试Flash Attention

3. **高级练习**:
   - 实现模型并行
   - 构建优化pipeline
   - 进行端到端性能优化

---

## 📚 参考资源

- [PyTorch性能优化](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html)
- [HuggingFace优化文档](https://huggingface.co/docs/transformers/perf_train_gpu_one)
- [BitsAndBytes文档](https://github.com/TimDettmers/bitsandbytes)
- [Flash Attention](https://github.com/Dao-AILab/flash-attention)

---

**🎓 恭喜完成性能优化实战教程!**